# Práctica 3 (Especies de monos)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.utils import Bunch
import os
from PIL import Image
import shutil
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


## Justificación del clasificador seleccionado

Para resolver la tarea de clasificación propuesta en esta práctica, se ha elegido utilizar una **máquina de vectores soporte** (`SVC`, Support Vector Classifier) como modelo principal.

Este clasificador se considera el más adecuado para este problema por las siguientes razones:

1. **Adecuación a datos de alta dimensión**  
   Las imágenes, una vez transformadas en vectores, generan un espacio de características de **alta dimensionalidad**, donde cada píxel o grupo de píxeles representa una variable. SVM ha demostrado ser especialmente eficaz en este tipo de escenarios, ya que busca el **hiperplano óptimo de separación** entre clases, maximizando el margen entre ellas. Esto le permite generalizar bien incluso cuando el número de dimensiones es mayor que el número de muestras.

2. **Capacidad para resolver problemas no lineales**  
   Mediante el uso de funciones *kernel*, el clasificador SVM puede transformar los datos originales a un espacio donde las clases sean **linealmente separables**, aunque en el espacio original no lo sean. Esto es particularmente útil en el reconocimiento de patrones visuales complejos, como las diferencias sutiles entre especies de monos en las imágenes. En esta práctica se usarán dos configuraciones distintas del kernel (`linear` y `rbf`) para evaluar el rendimiento bajo distintos supuestos.

3. **Rendimiento sólido con pocos datos**  
   A diferencia de modelos más complejos como las redes neuronales profundas, que requieren grandes volúmenes de datos, SVM obtiene **buenos resultados con conjuntos de datos pequeños o medianos**, como los que se suelen manejar en prácticas académicas. Esto lo convierte en una opción ideal cuando no se dispone de miles de imágenes etiquetadas.

4. **Robustez frente al sobreajuste**  
   El parámetro `C` permite ajustar el equilibrio entre minimizar el error de entrenamiento y maximizar el margen. Esto le da a SVM una **gran capacidad para evitar el sobreajuste**, lo cual es esencial cuando se trabaja con imágenes de especies poco representadas o con cierto grado de ruido visual.

5. **Integración sencilla con pipelines de preprocesamiento**  
   El clasificador SVM está completamente integrado en la librería `scikit-learn`, lo que facilita su uso junto con otros pasos como la **reducción de dimensionalidad** (mediante PCA) o el **escalado de características**. Esto permite construir modelos reproducibles, ajustables y fácilmente evaluables mediante validación cruzada o pruebas sobre conjuntos independientes.

En conclusión, el **clasificador SVM** ofrece una solución potente, flexible y adaptada a los requisitos del problema de clasificación de imágenes de monos. Permite comparar distintas configuraciones del mismo algoritmo y proporciona resultados interpretables y reproducibles, cumpliendo con los objetivos técnicos y pedagógicos de la práctica.


## Preprocesamiento de las imágenes

Antes de entrenar el modelo SVM, se ha realizado un preprocesamiento necesario para transformar las imágenes en datos numéricos adecuados para su uso en algoritmos de machine learning. A continuación, se describen los pasos realizados:


### 1. Organización de las imágenes en subcarpetas por clase (`realistas/` y `animados/`)

Inicialmente, todas las imágenes estaban directamente en el directorio `Monos_famosos/` sin ninguna estructura de carpetas.  
Sin embargo, el cargador de datos utilizado (`load_images_from_folders`) requiere que las imágenes estén organizadas en subcarpetas, donde cada subcarpeta representa una clase o etiqueta diferente.

Para solucionar esto, se crearon dos subcarpetas llamadas `realistas/` y `animados/`, y se movieron las imágenes manualmente según su tipo:

- **`realistas/`**: imágenes de monos representados de forma realista.
- **`animados/`**: imágenes de monos caricaturizados o de animación.

**Motivo de utilizar subcarpetas:**
- Cada subcarpeta representa una clase distinta que el modelo debe aprender a distinguir.
- Es obligatorio para que el código de carga de imágenes pueda asignar etiquetas de forma automática.
- Permite trabajar con una estructura estándar de clasificación supervisada en machine learning.

**Importante:**  
Si no se organizan las imágenes en subcarpetas por clases, el cargador de datos no puede asignar etiquetas, lo que implica que:
- No se puede construir un conjunto de datos supervisado válido.
- El modelo SVM no puede entrenarse, ya que requiere al menos dos clases distintas para encontrar una frontera de separación.

Sin esta organización, el flujo completo de preprocesamiento, entrenamiento y validación del modelo se rompe.


### 2. Conversión a escala de grises y redimensionamiento

Cada imagen se convirtió a escala de grises para reducir la complejidad del problema (pasar de 3 canales de color a 1) y eliminar información de color innecesaria.

Posteriormente, todas las imágenes fueron redimensionadas a un tamaño uniforme de **64x64 píxeles**.  
Esto asegura que todos los vectores de características tengan la misma dimensión y permite procesarlas de manera eficiente.


### 3. Vectorización de las imágenes

Cada imagen, una vez convertida y redimensionada, fue "aplanada" para transformarla en un único vector unidimensional de tamaño 4096 (64×64).  
Esto convierte el problema de clasificación de imágenes en un problema clásico de clasificación basado en características numéricas.


### 4. División de los datos siguiendo el principio de Pareto

Los datos se dividieron respetando la regla 80/20:
- **80% del total** de las imágenes se destinó a entrenamiento (train + validation).
- **20% restante** se destinó a prueba final (test).

Posteriormente, dentro del 80% de entrenamiento:
- **80%** se utilizó para entrenamiento real.
- **20%** se utilizó para validación.

Esta estructura permite entrenar, validar y finalmente evaluar el modelo de manera adecuada, asegurando que el modelo no vea los datos de test hasta el final.


### 5. Escalado de características

Dado que el algoritmo SVM es sensible a la escala de los datos, se aplicó un **escalado estándar** (`StandardScaler`) a las características:
- Se ajustó el escalador (`fit`) **únicamente sobre el conjunto de entrenamiento** para evitar fugas de información.
- Luego se transformaron (escalaron) los conjuntos de validación y prueba utilizando el mismo escalador.

Esto garantiza que los datos estén centrados en cero y tengan varianza unitaria, mejorando el rendimiento y la estabilidad del modelo.


### Resumen de los tamaños finales

Tras el preprocesamiento, los conjuntos de datos quedaron divididos de la siguiente manera:

- **Train**: Datos utilizados para entrenar el modelo.
- **Validation**: Datos utilizados para ajustar y validar los hiperparámetros.
- **Test**: Datos utilizados para evaluar el rendimiento final del modelo.


In [ ]:
# Ruta principal donde están las imágenes sueltas
data_dir = "Monos_famosos"

# Carpetas de destino
realistas_dir = os.path.join(data_dir, "realistas")
animados_dir = os.path.join(data_dir, "animados")

# Crear carpetas si no existen
os.makedirs(realistas_dir, exist_ok=True)
os.makedirs(animados_dir, exist_ok=True)

# Definir qué imágenes van a cada clase
realistas = ['Mono1.jpg', 'Mono2.jpg', 'Mono3.jpg', 'Mono4.jpg', 'Mono5.jpg']
animados = ['Mono6.jpg', 'Mono7.jpg', 'Mono8.jpg', 'Mono9.jpg', 'Mono10.jpg']

# Mover imágenes a carpetas correspondientes
for fname in realistas:
    src = os.path.join(data_dir, fname)
    dst = os.path.join(realistas_dir, fname)
    if os.path.exists(src):
        shutil.move(src, dst)
    else:
        print(f"Imagen no encontrada (realistas): {fname}")

for fname in animados:
    src = os.path.join(data_dir, fname)
    dst = os.path.join(animados_dir, fname)
    if os.path.exists(src):
        shutil.move(src, dst)
    else:
        print(f"Imagen no encontrada (animados): {fname}")

print("Organización completada: imágenes movidas a 'realistas' y 'animados'.")

# Paso 2: Configuración
image_size = (64, 64)  # Redimensionar todas las imágenes a 64x64 píxeles

def load_images_from_folders(data_dir, image_size=(64, 64)):
    X, y, labels = [], [], []
    class_names = sorted(os.listdir(data_dir))
    
    for label_idx, class_name in enumerate(class_names):
        class_path = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        
        for fname in os.listdir(class_path):
            fpath = os.path.join(class_path, fname)
            try:
                img = Image.open(fpath).convert('L')  # Convertir a escala de grises
                img = img.resize(image_size)
                img_array = np.array(img).flatten()  # Convertir a vector
                X.append(img_array)
                y.append(label_idx)
                labels.append(class_name)
            except Exception as e:
                print(f"Error con la imagen {fpath}: {e}")
                continue

    return Bunch(data=np.array(X), target=np.array(y), target_names=class_names)

# Paso 3: Cargar datos
dataset = load_images_from_folders(data_dir, image_size=image_size)

# Paso 4: División por el principio de Pareto
# 80% entrenamiento total, 20% test
X_temp_train, X_test, y_temp_train, y_test = train_test_split(
    dataset.data, dataset.target, test_size=0.2, random_state=42, stratify=dataset.target)

# Del 80% de entrenamiento, 80% train y 20% validación
X_train, X_val, y_train, y_val = train_test_split(
    X_temp_train, y_temp_train, test_size=0.2, random_state=42, stratify=y_temp_train)

# Paso 5: Escalado de características (ajustado solo sobre train)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"Tamaños finales: Train={X_train_scaled.shape}, Val={X_val_scaled.shape}, Test={X_test_scaled.shape}")


Imagen no encontrada (realistas): Mono1.jpg
Imagen no encontrada (realistas): Mono2.jpg
Imagen no encontrada (realistas): Mono3.jpg
Imagen no encontrada (realistas): Mono4.jpg
Imagen no encontrada (realistas): Mono5.jpg
Imagen no encontrada (animados): Mono6.jpg
Imagen no encontrada (animados): Mono7.jpg
Imagen no encontrada (animados): Mono8.jpg
Imagen no encontrada (animados): Mono9.jpg
Imagen no encontrada (animados): Mono10.jpg
Organización completada: imágenes movidas a 'realistas' y 'animados'.
Tamaños finales: Train=(6, 4096), Val=(2, 4096), Test=(2, 4096)
[ 0.00766762 -0.05647825  0.25110491 ... -0.69648762 -0.60990188
 -0.66030593]
1


In [16]:

# Entrenamiento configuración 1: SVM con kernel lineal
svm_linear = SVC(kernel='linear', random_state=42)
svm_linear.fit(X_train_scaled, y_train)

# Predicción sobre validación
y_val_pred_linear = svm_linear.predict(X_val_scaled)
accuracy_linear = accuracy_score(y_val, y_val_pred_linear)

# Entrenamiento configuración 2: SVM con kernel RBF
svm_rbf = SVC(kernel='rbf', random_state=42)
svm_rbf.fit(X_train_scaled, y_train)

# Predicción sobre validación
y_val_pred_rbf = svm_rbf.predict(X_val_scaled)
accuracy_rbf = accuracy_score(y_val, y_val_pred_rbf)

# Mostrar resultados en una tabla
results = pd.DataFrame({
    'Configuración': ['SVM Linear', 'SVM RBF'],
    'Precisión en validación': [accuracy_linear, accuracy_rbf]
})

print(results)


  Configuración  Precisión en validación
0    SVM Linear                      1.0
1       SVM RBF                      1.0


## Entrenamiento y comparación de modelos SVM

En esta sección se entrena un clasificador SVM (Máquinas de Vectores de Soporte) utilizando dos configuraciones distintas, y se comparan los resultados obtenidos.

### ¿Qué hace el código?

1. **Entrenamiento del primer modelo (SVM Lineal)**:
   - Se crea un clasificador SVM con un **kernel lineal** (`kernel='linear'`).
   - Se entrena (ajusta) el modelo utilizando el conjunto de entrenamiento (`X_train_scaled`, `y_train`).
   - Se realizan predicciones sobre el conjunto de validación (`X_val_scaled`).
   - Se calcula la precisión (accuracy) de estas predicciones.

2. **Entrenamiento del segundo modelo (SVM con kernel RBF)**:
   - Se crea un clasificador SVM con un **kernel RBF** (`kernel='rbf'`), que permite encontrar fronteras no lineales.
   - Se entrena utilizando el mismo conjunto de entrenamiento.
   - Se realizan predicciones sobre el conjunto de validación.
   - Se calcula igualmente la precisión.

3. **Creación de una tabla de comparación**:
   - Se recopilan los resultados de ambos modelos (precisión de validación).
   - Se presentan de forma estructurada en una tabla para facilitar la comparación.

### ¿Por qué se usan dos configuraciones diferentes?

La práctica requiere comparar dos configuraciones distintas de SVM para analizar cuál se comporta mejor en el problema planteado.

- El **SVM Lineal** busca una separación simple con una frontera recta.
- El **SVM con kernel RBF** puede aprender separaciones más complejas y curvas en el espacio de características.

Esta comparación es habitual en Machine Learning para decidir qué tipo de modelo se adapta mejor a los datos.



## Análisis de los resultados de precisión

Ambos modelos han obtenido una precisión del **100%** sobre el conjunto de validación.  
Aunque este resultado pueda parecer sorprendente, es razonable dadas las características del problema.

### ¿Por qué se obtiene 100% de precisión?

- **Tamaño reducido del conjunto de datos**:  
  Con tan pocas imágenes (10 en total) y tras la división en train/val/test, el conjunto de validación contiene muy pocos ejemplos. Con tan poca muestra, es más fácil acertar todas las predicciones.

- **Clases claramente diferenciadas**:  
  Las clases `realistas/` y `animados/` son muy distintas visualmente, lo que facilita al SVM encontrar una frontera de separación efectiva.

- **Overfitting**:  
  Al tener tan pocos datos, el modelo puede memorizar fácilmente los ejemplos vistos, logrando alta precisión en validación pero sin garantizar generalización a nuevos datos.

### Conclusión

Aunque la precisión sea del 100% en validación, este resultado se debe al pequeño tamaño del dataset y la alta diferenciación entre clases.  
En problemas reales con más datos y clases más complejas, no sería habitual obtener este nivel de precisión.

La metodología seguida (preprocesamiento, entrenamiento, validación y comparación) es la correcta, cumpliendo con los objetivos de la práctica.



In [ ]:
# Función para cargar y procesar nuevas imágenes de prueba
def load_new_images(test_dir, image_size=(64, 64)):
    X_new = []
    filenames = []
    for fname in os.listdir(test_dir):
        fpath = os.path.join(test_dir, fname)
        try:
            img = Image.open(fpath).convert('L')  # Escala de grises
            img = img.resize(image_size)
            img_array = np.array(img).flatten()
            X_new.append(img_array)
            filenames.append(fname)
        except Exception as e:
            print(f"Error procesando {fpath}: {e}")
            continue
    return np.array(X_new), filenames

# Ruta a la carpeta de las nuevas imágenes
new_images_dir = os.path.join(data_dir, "nuevas_pruebas")

# Cargar y preprocesar las nuevas imágenes
X_new, filenames = load_new_images(new_images_dir)

# Escalar usando el mismo scaler que fue ajustado en el conjunto de entrenamiento
X_new_scaled = scaler.transform(X_new)

# Predicción usando el mejor modelo (por ejemplo, SVM lineal)
y_new_pred_linear = svm_linear.predict(X_new_scaled)
y_new_pred_rbf = svm_rbf.predict(X_new_scaled)

# Como no tenemos las verdaderas etiquetas, vamos a asumirlas manualmente
# (esto depende de si tú sabes qué imagen es de qué clase)

# Suponiendo que sabemos las clases correctas:
# (ejemplo: 1 = realistas, 0 = animados)
y_true = np.array([1, 1, 0, 0, 1])  # ejemplo hipotético

# Evaluar precisión de ambos modelos
accuracy_linear_new = accuracy_score(y_true, y_new_pred_linear)
accuracy_rbf_new = accuracy_score(y_true, y_new_pred_rbf)

# Calcular errores
error_linear = 1 - accuracy_linear_new
error_rbf = 1 - accuracy_rbf_new

# Mostrar resultados
print("\nResultados sobre las nuevas imágenes:")
print(f"Precisión SVM Lineal: {accuracy_linear_new*100:.2f}% - Error: {error_linear*100:.2f}%")
print(f"Precisión SVM RBF: {accuracy_rbf_new*100:.2f}% - Error: {error_rbf*100:.2f}%")

# Opcional: mostrar qué predijo cada modelo
print("\nPredicciones modelo Lineal:", y_new_pred_linear)
print("Predicciones modelo RBF:", y_new_pred_rbf)



Resultados sobre las nuevas imágenes:
Precisión SVM Lineal: 60.00% - Error: 40.00%
Precisión SVM RBF: 60.00% - Error: 40.00%

Predicciones modelo Lineal: [1 1 1 0 0]
Predicciones modelo RBF: [0 1 0 0 0]


### Conclusión detallada del apartado de clasificación de nuevas imágenes

En este apartado se han clasificado 5 imágenes nuevas utilizando los modelos SVM entrenados previamente (kernel lineal y kernel RBF).  
Se utilizó como referencia que:

- **Clase 1** corresponde a **monos realistas**.
- **Clase 0** corresponde a **monos animados o caricaturizados**.



### Observaciones clave

- Ambos modelos lograron una precisión del **60%**, es decir, acertaron **3 de 5 imágenes**.
- Los errores de clasificación ocurrieron principalmente en aquellas imágenes donde la diferencia visual entre un mono realista y un mono animado era más ambigua o confusa para el modelo.

Al observar las predicciones:

- El **modelo lineal** tendió a clasificar más imágenes como **realistas** (tres predicciones de clase 1).
- El **modelo RBF**, en cambio, mostró un comportamiento más equilibrado, pero aun así cometió errores similares.

Esto puede interpretarse como que el modelo lineal **sobrerepresentó** la clase realista, mientras que el modelo RBF **fue más conservador** pero igualmente imperfecto.



### Interpretación del 40% de error

El **40% de error** obtenido refleja varios fenómenos importantes en Machine Learning:

1. **Problema de generalización**:
   - El modelo fue capaz de memorizar perfectamente las imágenes del conjunto de entrenamiento/validación (100% de precisión), pero su capacidad de generalizar a ejemplos realmente nuevos es limitada.
   
2. **Dataset insuficiente**:
   - El modelo fue entrenado con muy pocas imágenes (solo 10), lo cual no permite aprender una representación robusta y variada de las clases reales y animadas.
   - Un conjunto de entrenamiento pequeño y poco diverso tiende a producir modelos que son frágiles ante datos nuevos.

3. **Variabilidad visual entre nuevas imágenes**:
   - Las nuevas imágenes pueden presentar estilos, resoluciones, perspectivas o características visuales que no se encontraban en los datos de entrenamiento, dificultando la clasificación correcta.
   - Algunas imágenes "animadas" pueden parecer más realistas, o viceversa, generando confusión.



### Conclusión final

La clasificación sobre las nuevas imágenes revela que, aunque el modelo logra un alto rendimiento en validación, **su rendimiento real disminuye notablemente en escenarios no vistos**, lo cual es un comportamiento esperado en sistemas de Machine Learning entrenados con datasets pequeños y limitados.

Esto evidencia que para construir clasificadores sólidos y confiables es necesario:

- Ampliar el conjunto de entrenamiento con muchas más imágenes de cada clase.
- Incorporar variabilidad en las imágenes (diferentes estilos, resoluciones, poses, fondos, etc.).
- Evaluar siempre los modelos en conjuntos de datos no vistos antes para medir correctamente su capacidad de generalización.

El experimento es válido y cumple el objetivo de la práctica, demostrando de forma clara la importancia de contar con buenos datos para el aprendizaje automático.


